In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
import Packages.CoNLLReader as reader
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [2]:
text, data = reader.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,tags,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",,,,,,
1,0,rejects,"(3, 10)",,,,,,
2,0,German,"(11, 17)",B,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",,,,,,
4,0,to,"(23, 25)",,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1390, 1397)",,,,,,
285585,1392,brother,"(1398, 1405)",,,,,,
285586,1392,",","(1406, 1407)",,,,,,
285587,1392,Bobby,"(1408, 1413)",B,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [3]:
# Gold standard
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['mentions'].values
docs = ents_data['documents'].values
unique_entities = np.unique(entities)
gold_standard = dict.fromkeys(entities, 0)
for uniqu_ent in tqdm(entities):
    gold_standard[uniqu_ent] = gold_standard[uniqu_ent] + 1
ent_nums = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

100%|██████████| 27817/27817 [00:00<00:00, 1981302.40it/s]


In [7]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0.txt", dtype=np.int32)
# Now we create a dict for each cluster that contains entities and entities count
lev_cluster_dict = dict.fromkeys(np.unique(leven_cluster), [])
for i, x in enumerate(leven_cluster):
    lev_cluster_dict[x].append(mentions[i])

# """se uso un cluster diverso da minimo 0 allora scommento"""
# del lev_cluster_dict[-1]

In [8]:
for key in tqdm(lev_cluster_dict):
    cluster_list = [mention_to_entity[x] for x in lev_cluster_dict[key]]
    cluster_set = set(cluster_list)
    cluster_dict = dict.fromkeys(list(cluster_set), 0)
    for cluster in cluster_list:
        cluster_dict[cluster] = cluster_dict[cluster] + 1
    lev_cluster_dict[key] = cluster_dict

100%|██████████| 6066/6066 [01:28<00:00, 68.35it/s]


### Bag of Words

In [ ]:
# creation of BOW

CountVec = CountVectorizer(ngram_range=(1, 1), stop_words='english', lowercase=True, strip_accents="ascii",
                           analyzer="word")
#transform
bow_text = CountVec.fit_transform(text)
bow_df = pd.DataFrame(bow_text.toarray(), columns=CountVec.get_feature_names())
bow_vectors = bow_text.toarray()

In [42]:
text_coupled = [(text[i], bag) for i, bag in enumerate(bow_vectors)]

{'"',
 "'",
 "'s",
 '(',
 ')',
 ',',
 '--',
 '.',
 '10',
 '1996-08-22',
 '4,275',
 '47,600',
 'BBC',
 'BRUSSELS',
 'BSE',
 'Blackburn',
 'Bonn',
 'Bovine',
 'Britain',
 'British',
 'But',
 'Commission',
 'Commissioner',
 'EU',
 'EU-wide',
 'Encephalopathy',
 'Europe',
 'European',
 'Farm',
 'Farmers',
 'Fischler',
 'France',
 'Franz',
 'German',
 'Germany',
 'He',
 'It',
 'John',
 'Jones',
 'Lloyd',
 'Loyola',
 'March',
 'Minister',
 'NFU',
 'National',
 'Nikolaus',
 'Only',
 'Palacio',
 'Pas',
 'Peter',
 'Sheep',
 'Spanish',
 'Spongiform',
 'The',
 'Thursday',
 'Union',
 'We',
 'Wednesday',
 'Welsh',
 'Werner',
 'What',
 'Zwingmann',
 'a',
 'accused',
 'across',
 'action',
 'advice',
 'after',
 'agreed',
 'alarm',
 'an',
 'and',
 'animal',
 'any',
 'are',
 'as',
 'at',
 'avoid',
 'backed',
 'ban',
 'be',
 'because',
 'beef',
 'been',
 'believed',
 'boycott',
 'brain-wasting',
 'brains',
 'briefing',
 'brought',
 'but',
 'buy',
 'by',
 'call',
 'can',
 'careful',
 'cattle',
 'causing',

136